In [1]:
import pandas as pd
import urllib3 as urllib
import json
import qgrid
from IPython.display import JSON

In [2]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [9]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [10]:
df = pd.DataFrame(ltc_data['FacilityValues'])
df['reporting_date'] = reporting_date
df['% deaths'] = df['deaths'] / df['confirmed_cases']

In [11]:
qgrid.show_grid(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [12]:
a = df.sum()
print ('Date: %s' % reporting_date)
print ('Cases: %d' % a.confirmed_cases)
print ('Deaths:  %d' % a.deaths)

Date: 2020-05-29
Cases: 17135
Deaths:  2747


In [13]:
df_county = df.groupby(by=['County']).sum()
df_county['% deaths'] = df_county['deaths'] / df_county['confirmed_cases']
df_county.sort_values('% deaths', ascending=False)

,confirmed_cases,deaths,% deaths
County,,,
Tazewell,6,2,0.333333
Kendall,50,16,0.320000
Jackson,36,10,0.277778
Monroe,42,11,0.261905
Madison,189,47,0.248677
LaSalle,41,9,0.219512
Jefferson,73,15,0.205479
Winnebago,147,30,0.204082
Macon,98,19,0.193878


In [14]:
ltc_data

{'LastUpdateDate': {'year': 2020, 'month': 5, 'day': 29},
 'LTC_Reported_Cases': {'confirmed_cases': 17133, 'deaths': 2747},
 'LTC_Historical_Reported_Cases': [{'reported_date': '5/29/2020',
   'confirmed_cases': 17133,
   'deaths': 2747},
  {'reported_date': '5/23/2020', 'confirmed_cases': 14882, 'deaths': 2402}],
 'FacilityValues': [{'County': 'Boone',
   'FacilityName': 'Park Place of Belvidere',
   'confirmed_cases': 5,
   'deaths': 0,
   'status': 'Open'},
  {'County': 'Boone',
   'FacilityName': 'Symphony Northwoods',
   'confirmed_cases': 84,
   'deaths': 11,
   'status': 'Open'},
  {'County': 'Champaign',
   'FacilityName': 'Reflections Memory Care',
   'confirmed_cases': 8,
   'deaths': 0,
   'status': 'Open'},
  {'County': 'Champaign',
   'FacilityName': 'University Rehab Center',
   'confirmed_cases': 5,
   'deaths': 0,
   'status': 'Open'},
  {'County': 'Christian',
   'FacilityName': 'Villas of Holly Brook',
   'confirmed_cases': 2,
   'deaths': 0,
   'status': 'Open'},
  